### If there is a set of 10 positive numbers, are there two subsets that sum of the elements in both sets is equal?

#### objective function
$$minimize \sum_{i=0}^{10} n_{i}s_{i}$$

In [118]:
from pyqubo import Array
s=Array.create('s',shape=10,vartype='SPIN')   
H=sum(n*s for s,n in zip(s,range(1,11)))**2
model=H.compile()
qubo,offset=model.to_qubo()

In [119]:
bqm=model.to_dimod_bqm()

In [128]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
sampler=EmbeddingComposite(DWaveSampler())
response=sampler.sample(bqm)

In [129]:
for sample in response.samples():
     print(sample)

{'s[0]': 1, 's[1]': 1, 's[2]': 1, 's[3]': 0, 's[4]': 1, 's[5]': 0, 's[6]': 0, 's[7]': 0, 's[8]': 1, 's[9]': 1}


In [130]:
import pandas as pd
df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
df['level_0']=df['level_0']+1
ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},
    'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==1]['level_0']))}}

In [131]:
print(pd.DataFrame.from_dict(ans))

            group1              group2:
list  [4, 6, 7, 8]  [1, 2, 3, 5, 9, 10]
sum             25                   30


### Asking for more samples

In [132]:
response=sampler.sample(bqm,num_reads=10)

In [133]:
for sample in response.samples():
    df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
    df['level_0']=df['level_0']+1
    ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==1]['level_0']))}}
    print(pd.DataFrame.from_dict(ans))

            group1              group2:
list  [3, 4, 7, 9]  [1, 2, 5, 6, 8, 10]
sum             23                   32
             group1             group2:
list  [1, 3, 4, 10]  [2, 5, 6, 7, 8, 9]
sum              18                  37
             group1             group2:
list  [1, 3, 4, 10]  [2, 5, 6, 7, 8, 9]
sum              18                  37
            group1              group2:
list  [1, 2, 6, 8]  [3, 4, 5, 7, 9, 10]
sum             17                   38
          group1                group2:
list  [3, 4, 10]  [1, 2, 5, 6, 7, 8, 9]
sum           17                     38
               group1           group2:
list  [1, 2, 3, 4, 6]  [5, 7, 8, 9, 10]
sum                16                39
          group1                group2:
list  [1, 4, 10]  [2, 3, 5, 6, 7, 8, 9]
sum           15                     40
               group1           group2:
list  [1, 2, 3, 4, 5]  [6, 7, 8, 9, 10]
sum                15                40
       group1                   group2:


### Add constraint to balance the size of two sets

In [144]:
from pyqubo import Constraint
M=5
H=H+M*Constraint(sum(s for s in s)**2,label='(sum s)^2= 0')

In [145]:
model=H.compile()
qubo,offset=model.to_qubo()
bqm=model.to_dimod_bqm()

In [146]:
sampler=EmbeddingComposite(DWaveSampler())
response=sampler.sample(bqm)

In [147]:
for sample in response.samples():
     print(sample)

{'s[0]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 1, 's[4]': 1, 's[5]': 1, 's[6]': 0, 's[7]': 1, 's[8]': 0, 's[9]': 0}


In [148]:
for sample in response.samples():
    df=pd.DataFrame.from_dict(sample,orient='index').reset_index().reset_index()
    df['level_0']=df['level_0']+1
    ans={'group1':{'list':list(df[df[0]==0]['level_0']),'sum':sum(list(df[df[0]==0]['level_0']))},'group2:':{'list':list(df[df[0]==1]['level_0']),'sum':sum(list(df[df[0]==1]['level_0']))}}
    print(pd.DataFrame.from_dict(ans))

             group1             group2:
list  [1, 7, 9, 10]  [2, 3, 4, 5, 6, 8]
sum              27                  28
